# Job 1 optimized version

In [1]:
import org.apache.spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.195:4041
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1757948759257)
SparkSession available as 'spark'


import org.apache.spark


### Schema definitions

In [2]:
import org.apache.spark.sql.types._
import java.sql.Timestamp

val reviewSchema = StructType(
  Seq(
    StructField("user_id",  StringType,            nullable = true),
    StructField("name",     StringType,            nullable = true),
    StructField("time",     LongType,              nullable = false),
    StructField("rating",   DoubleType,            nullable = true),
    StructField("text",     StringType,            nullable = true),
    StructField("pics",     ArrayType(StringType), nullable = true),
    StructField("resp",     StructType(
      Seq(
        StructField("time", LongType,              nullable = false),
        StructField("text", StringType,            nullable = true)
      )
    ),                                             nullable = true),
    StructField("gmap_id",  StringType,            nullable = false),
  )
)

case class Response(time: Timestamp, text: Option[String])

case class Review(
  user_id: Option[String],
  name: Option[String],
  time: Timestamp,
  rating: Option[Double],
  text: Option[String],
  pics: Seq[String],
  resp: Option[Response],
  gmap_id: String
)

import org.apache.spark.sql.types._
import java.sql.Timestamp
reviewSchema: org.apache.spark.sql.types.StructType = StructType(StructField(user_id,StringType,true),StructField(name,StringType,true),StructField(time,LongType,false),StructField(rating,DoubleType,true),StructField(text,StringType,true),StructField(pics,ArrayType(StringType,true),true),StructField(resp,StructType(StructField(time,LongType,false),StructField(text,StringType,true)),true),StructField(gmap_id,StringType,false))
defined class Response
defined class Review


In [3]:
val metadataSchema = StructType(
  Seq(
    StructField("name",             StringType,                                 nullable = true),
    StructField("address",          StringType,                                 nullable = true),
    StructField("gmap_id",          StringType,                                 nullable = false),
    StructField("description",      StringType,                                 nullable = true),
    StructField("latitude",         DoubleType,                                 nullable = false),
    StructField("longitude",        DoubleType,                                 nullable = false),
    StructField("category",         ArrayType(StringType),                      nullable = true),
    StructField("avg_rating",       DoubleType,                                 nullable = false),
    StructField("num_of_reviews",   IntegerType,                                nullable = false),
    StructField("price",            StringType,                                 nullable = true),
    StructField("hours",            ArrayType(ArrayType(StringType)),           nullable = true),
    StructField("MISC",             MapType(StringType, ArrayType(StringType)), nullable = true),
    StructField("state",            StringType,                                 nullable = true),
    StructField("relative_results", ArrayType(StringType),                      nullable = true),
    StructField("url",              StringType,                                 nullable = false),
  )
)

case class Metadata(
  name: Option[String],
  address: Option[String],
  gmap_id: String,
  description: Option[String],
  latitude: Double,
  longitude: Double,
  category: Seq[String],
  avg_rating: Double,
  num_of_reviews: Int,
  price: Option[String],
  hours: Seq[Seq[String]],
  MISC: Map[String, Seq[String]],
  state: Option[String],
  relative_results: Seq[String],
  url: String
)

metadataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true),StructField(address,StringType,true),StructField(gmap_id,StringType,false),StructField(description,StringType,true),StructField(latitude,DoubleType,false),StructField(longitude,DoubleType,false),StructField(category,ArrayType(StringType,true),true),StructField(avg_rating,DoubleType,false),StructField(num_of_reviews,IntegerType,false),StructField(price,StringType,true),StructField(hours,ArrayType(ArrayType(StringType,true),true),true),StructField(MISC,MapType(StringType,ArrayType(StringType,true),true),true),StructField(state,StringType,true),StructField(relative_results,ArrayType(StringType,true),true),StructField(url,StringType,false))
defined class Metadata


### Dataset load and parse

In [4]:
import java.nio.file.Paths

val projectDir: String = Paths.get(System.getProperty("user.dir")).getParent.getParent.getParent.toString
val reviewsPath = s"$projectDir/dataset/sample-reviews.ndjson"
val metadataPath = s"$projectDir/dataset/metadata.ndjson"

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("NDJSON Reader")
  .master("local[*]") // Needed in local mode
  .getOrCreate()

val reviewsDf = spark.read
  .schema(reviewSchema)
  .json(reviewsPath)
  .withColumn("pics", when (col("pics") isNull, array()) otherwise col("pics"))
  .withColumn("time", from_unixtime(col("time") / 1000).cast("timestamp"))
  .withColumn("resp", 
    when (
      col("resp") isNotNull, 
      struct(
        from_unixtime(col("resp.time") / 1000).cast("timestamp").alias("time"),
        col("resp.text").cast(StringType).alias("text")
      )
    ) otherwise lit(null)
  )
  .as[Review]

val metadataDf = spark.read
  .schema(metadataSchema)
  .json(metadataPath)
  .withColumn("category", when (col("category") isNull, array()) otherwise col("category"))
  .withColumn("hours", when (col("hours") isNull, array()) otherwise col("hours"))
  .withColumn("relative_results", when (col("relative_results") isNull, array()) otherwise col("relative_results"))
  .withColumn("MISC",
    when (
      col("MISC") isNotNull,
      col("MISC").cast(MapType(StringType, ArrayType(StringType)))
    ) otherwise typedLit(Map.empty[String, Seq[String]])
  )
  .as[Metadata]

reviewsDf.printSchema()
metadataDf.printSchema()

// Unforturnately, it seems that Spark does not support case classes in RDDs. It throws ArrayStoreException
// when trying to collect the RDD... [see also [here](https://github.com/adtech-labs/spylon-kernel/issues/40)]
val reviewsRdd = reviewsDf.rdd
  .map(Review.unapply)
  .map(_.get)
  .map { case review @ (_, _, _, _, _, _, resp, _) => review.copy(_7 = resp.map(Response.unapply(_).get)) }
val metaRdd = metadataDf.rdd.map(Metadata.unapply).map(_.get)

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resp: struct (nullable = true)
 |    |-- time: timestamp (nullable = true)
 |    |-- text: string (nullable = true)
 |-- gmap_id: string (nullable = true)

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- avg_rating: double (nullable = true)
 |-- num_of_reviews: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- hours: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: 

import java.nio.file.Paths
projectDir: String = /Users/lucatassi/Projects/big-data/big-data-project
reviewsPath: String = /Users/lucatassi/Projects/big-data/big-data-project/dataset/sample-reviews.ndjson
metadataPath: String = /Users/lucatassi/Projects/big-data/big-data-project/dataset/metadata.ndjson
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1d6da03c
reviewsDf: org.apache.spark.sql.Dataset[Review] = [user_id: string, name: string ... 6 more fields]
metadataDf: org.apache.spark.sql.Dataset[Metadata] = [name: string, address: string ... 13 more fields]
reviewsRdd: org.apache.spark.rdd.RDD[(Option[String], Option[String], java.sql.Timestamp, Option[Double], Option[String], Seq[String], Option[(java.sql.Timestamp, ...


### Optimized version

The goal of this job is to understand, year by year, whether greater frequency in responding to reviews has an impact on the average rating received.

Specifically:

- For each year and business, the average rating, rate, and average response time are calculated;
- Based on the rate and average response time, an additional attribute "response strategy" is calculated that categorizes the business in a particular year into four categories ("Rapid and frequent," "Slow but frequent," "Occasional," or "Rare or none");
- Aggregation based on the "response strategy," year, and state to get the average rate and number of businesses within the category.

---

**Metadata**: (name, address, <ins>gmap_id</ins>, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url)

**Review**: (user_id, name, time, rating, text, pics, responses, <ins>gmap_id</ins>)

---

In [5]:
import java.util.concurrent.TimeUnit
import org.apache.spark.sql.SaveMode
import org.apache.spark.storage.StorageLevel

import java.util.concurrent.TimeUnit
import org.apache.spark.sql.SaveMode
import org.apache.spark.storage.StorageLevel


In [7]:
/**
 * Determines the response strategy, namely a string-based indicator of how businesses respond to reviews based on the
 * provided `avgResponseRate` and `avgResponseTime`.
 */
def responseStrategy(avgResponseRate: Double, avgResponseTime: Double): String =
  (avgResponseRate, avgResponseTime) match {
    case (rr, rt) if rr >= 0.5 && rt <= 4 * 24 => "Rapid and frequent"
    case (rr, rt) if rr >= 0.5 => "Slow but frequent"
    case (rr, _)  if rr >= 0.15 => "Occasional"
    case _ => "Rare or none"
  }

responseStrategy: (avgResponseRate: Double, avgResponseTime: Double)String


In [8]:
/** This regex captures the state abbreviation between a comma and the ZIP code. */
private val StateRegex = """,\s*([A-Z]{2})\s+\d{5}""".r

/** The map of states that are considered for the analysis. */
val consideredStates = Map(
  "Alabama" -> "AL",
  "Mississippi" -> "MS",
  "New Hampshire" -> "NH",
  "New Mexico" -> "NM",
  "Washington" -> "WA",
)

/** Regex to match state names in the address. */
val StateNameRegex = s"""\\b(${consideredStates.keys.mkString("|")})\\b""".r

/** Regex to match state abbreviations in the address. */
val StateAbbrevRegex = s"""\\b(${consideredStates.values.mkString("|")})\\b""".r

/**
 * Extracts the state from the given address.
 * @param address
 *   the optional address string
 * @return
 *   the state abbreviation or "Unknown" if no valid state is found
 * @see
 *   [[consideredStates]]
 */
def toState(address: Option[String]): String = address
  .flatMap { addr =>
    StateRegex
      .findFirstMatchIn(addr)
      .map(_.group(1))
      .orElse(StateNameRegex.findFirstMatchIn(addr).map(stateName => consideredStates(stateName.group(1))))
      .orElse(StateAbbrevRegex.findFirstMatchIn(addr).map(_.group(1)))
  }
  .filter(consideredStates.values.toSeq.contains)
  .getOrElse("Unknown")

consideredStates: scala.collection.immutable.Map[String,String] = Map(New Mexico -> NM, Washington -> WA, Alabama -> AL, Mississippi -> MS, New Hampshire -> NH)
StateNameRegex: scala.util.matching.Regex = \b(New Mexico|Washington|Alabama|Mississippi|New Hampshire)\b
StateAbbrevRegex: scala.util.matching.Regex = \b(NM|WA|AL|MS|NH)\b
toState: (address: Option[String])String


- Metadata: 292.5 MiB
- Reviews: 9.8 GiB

The "basic" (not optimized) job has 5 stages (c.f. `Job1 Basic`):

| #Stage | Input | Output | Shuffle read | Shuffle write | Duration | Partitions |
|--------|-------|--------|--------------|---------------|----------|------------|
| 0: load metadata | 292.5 MiB | | | 8.9 MiB | 12 s | 3 |
| 1: load reviews  | 9.8 GiB | | | 759.0 MiB | 1.6 min | 79 |
| 2: join datasets | | | 767.8 MiB | 69.3 MiB | 53s | 79 |
| 3: aggregate by key | | | 69.3 MiB | 2007.2 KiB | 3s | 79 | 
| 4: aggregate by key | | | 2007.2 KiB | 11.2 KiB | 2s | 1 |

**Total time: 2.6 minutes**

<ins>Optimizations:</ins>

- The most time-consuming stage, not considering the data loading, is the #2.
This can be optimized by **pushing down the join of the reviews only when the reviews have already been aggregated**, hence **reducing the amount of data shuffled** across the network;

    - Unfortunately, even if filtered, the metadata RDD is still too large, >42MiB, hence using broadcast join is not worth it.

- To further filter out not used data, only the timestamp of the response, if any, is kept during the first reviews aggregation phase.

- After the join the final `map` + `aggregateByKey` operations can be rewritten using `map` + `reduceByKey`;

In [9]:
val businessesStates = metaRdd.map(b => b._3 -> toState(b._2)) // [(gmap_id, state)*]

businessesStates: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[17] at map at <console>:35


In [10]:
val reviewsInfo = reviewsRdd
  .filter(_._4.isDefined) // filter out reviews without a rating
  .map { case (_, _, time, rating, _, _, resp, id) => 
    (time.toLocalDateTime.getYear, id) -> (time, rating.get, resp.map(_._1)) 
  }
  .aggregateByKey((0.0, 0, 0L, 0))( // (ratings sum, #responses, sum of response time differences, #reviews)
    (acc, v) => {
      val (sumRatings, numResponses, sumResponseTimes, totalReviews) = acc
      val (time, rating, response) = v
      (
        sumRatings + rating,
        numResponses + (if (response.isDefined) 1 else 0),
        sumResponseTimes + (if (response.isDefined) response.get.getTime - time.getTime else 0L),
        totalReviews + 1
      )
    },
    (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2, r1._3 + r2._3, r1._4 + r2._4)
  )
  .mapValues { case (sumRatings, numResponses, sumResponseTimes, totalReviews) => 
    val avgResponseRate = numResponses.toDouble / totalReviews
    val avgResponseTime = if (numResponses > 0) TimeUnit.MILLISECONDS.toHours(sumResponseTimes / numResponses) else Double.PositiveInfinity
    (sumRatings / totalReviews, avgResponseRate, avgResponseTime, responseStrategy(avgResponseRate, avgResponseTime))
  } // [((year, gmap_id), (avg_rating, response_rate, avg_response_time, response_strategy))*]

reviewsInfo: org.apache.spark.rdd.RDD[((Int, String), (Double, Double, Double, String))] = MapPartitionsRDD[21] at mapValues at <console>:53


In [11]:
val outcome = reviewsInfo
  .map { case ((year, id), (avgRating, _, _, responseStrategy)) => id -> (year, responseStrategy, avgRating) }
  .join(businessesStates) // [(gmap_id, ((year, response_strategy, avg_rating), state))*]
  .map { case (_, ((year, responseStrategy, avgRating), state)) => (year, state, responseStrategy) -> (avgRating, 1) }
  .reduceByKey((a, b) => (a._1 + b._1, a._2 + b._2))
  .mapValues { case (sumRatings, totalBusinesses) => sumRatings / totalBusinesses }

outcome: org.apache.spark.rdd.RDD[((Int, String, String), Double)] = MapPartitionsRDD[28] at mapValues at <console>:41


In [12]:
val outputDirPath = s"$projectDir/output"

outputDirPath: String = /Users/lucatassi/Projects/big-data/big-data-project/output


In [13]:
outcome.map { case ((year, state, responseStrategy), avgRating) => (year, state, responseStrategy, avgRating) }
  .coalesce(1)
  .toDF("year", "state", "response_strategy", "avg_rating")
  .write.format("csv").option("header", "true")
  .mode(SaveMode.Overwrite)
  .save(s"file://$outputDirPath/job1-optimized-without-partitioning-output")

Results (c.f. `Job1 Optimized without partitioning`):

| #Stage | Input | Output | Shuffle read | Shuffle write | Duration | Partitions |
|--------|-------|--------|--------------|---------------|----------|------------|
| 0: load metadata | 292.6 MiB | | | 8.9 MiB | 42s | 3 |
| 1: load reviews  | 9.8 GiB | | | **66.0 MiB** | 1.5 min | 79 |
| 2: aggregate by key | | | **66.0 MiB** | 56.2 MiB | **5s** | 79 |
| 3: join | | | 65.1 MiB | 1999.5 KiB | 3s | 79 | 
| 4: reduce by key | | | 1999.5 KiB | 11.2 KiB | 2s | 1 |

What we gained is the a drop in the times of the stage #2 thanks to the fact reviews data have been first aggregated and then shuffled: compared to the non-optimized job we reduced the shuffle read size from 767.8 MiB to 66.0 MiB ($\delta$ = 701.8 MiB) and the duration from 53 to 5 seconds.

Changing from `aggregateByKey` to `reduceByKey` does not produce observable performances increments.

**Total time**: 1.7 minutes, **53% faster**

<ins>Last optimization:</ins>
Partitioning by `gmap_id` in $N=18$ partitions.
$N = 18$ have been chosen empirically based on the cluster configuration and the tasks analysis from Spark UI since it provides a good balance between parallelism and overhead due to task scheduling.

Moreover, since after the first reviews aggregation the data passes from 9.8GiB to 66.0 MiB a `coalesce` operation is applied to reduce the number of partitions and optimize the subsequent stages to avoid excessive task scheduling overhead.

In [14]:
val partitions = 18
val partitioner = new org.apache.spark.HashPartitioner(partitions)

val businessesStates = metaRdd
  .map(b => b._3 -> toState(b._2)) // [(gmap_id, state)*]
  .partitionBy(partitioner)

partitions: Int = 18
partitioner: org.apache.spark.HashPartitioner = org.apache.spark.HashPartitioner@12
businessesStates: org.apache.spark.rdd.RDD[(String, String)] = ShuffledRDD[35] at partitionBy at <console>:40


In [15]:
val reviewsInfo = reviewsRdd
  .filter(_._4.isDefined) // filter out reviews without a rating
  .map { case (_, _, time, rating, _, _, resp, id) =>
    (time.toLocalDateTime.getYear, id) -> (time, rating.get, resp.map(_._1))
  }
  .aggregateByKey((0.0, 0, 0L, 0))( // (ratings sum, #responses, sum of response time differences, #reviews)
    (acc, v) => {
      val (sumRatings, numResponses, sumResponseTimes, totalReviews) = acc
      val (time, rating, response) = v
      (
        sumRatings + rating,
        numResponses + (if (response.isDefined) 1 else 0),
        sumResponseTimes + (if (response.isDefined) response.get.getTime - time.getTime else 0L),
        totalReviews + 1,
      )
    },
    (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2, r1._3 + r2._3, r1._4 + r2._4),
  )
  .mapValues { case (sumRatings, numResponses, sumResponseTimes, totalReviews) =>
    val avgResponseRate = numResponses.toDouble / totalReviews
    val avgResponseTime =
      if (numResponses > 0) TimeUnit.MILLISECONDS.toHours(sumResponseTimes / numResponses)
      else Double.PositiveInfinity
    (
      sumRatings / totalReviews,
      avgResponseRate,
      avgResponseTime,
      responseStrategy(avgResponseRate, avgResponseTime),
    )
  } // [((year, gmap_id), (avg_rating, response_rate, avg_response_time, response_strategy))*]
  .coalesce(partitions)

reviewsInfo: org.apache.spark.rdd.RDD[((Int, String), (Double, Double, Double, String))] = CoalescedRDD[40] at coalesce at <console>:66


In [16]:
val outcome = reviewsInfo
  .map { case ((year, id), (avgRating, _, _, responseStrategy)) => id -> (year, responseStrategy, avgRating) }
  .partitionBy(partitioner)
  .join(businessesStates) // [(gmap_id, ((year, response_strategy, avg_rating), state))*]
  .map { case (_, ((year, responseStrategy, avgRating), state)) => (year, state, responseStrategy) -> (avgRating, 1) }
  .reduceByKey((a, b) => (a._1 + b._1, a._2 + b._2))
  .mapValues { case (sumRatings, totalBusinesses) => sumRatings / totalBusinesses }

outcome: org.apache.spark.rdd.RDD[((Int, String, String), Double)] = MapPartitionsRDD[48] at mapValues at <console>:43


In [17]:
outcome
  .map { case ((year, state, responseStrategy), avgRating) => (year, state, responseStrategy, avgRating) }
  .coalesce(1)
  .toDF("year", "state", "response_strategy", "avg_rating")
  .write.format("csv").option("header", "true")
  .mode(SaveMode.Overwrite)
  .save(s"file://$outputDirPath/job1-optimized-output")

Finally (c.f. `Job1 Optimized`):

| #Stage | Input | Output | Shuffle read | Shuffle write | Duration | Partitions |
|--------|-------|--------|--------------|---------------|----------|------------|
| 0: load metadata | 292.7 MiB | | | 8.7 MiB | 16s | 20 |
| 1: load reviews  | 9.8 GiB | | | 66.0 MiB | 1.5 min | 79 |
| 2: aggregate by key | | | 66.0 MiB | 54.6 MiB | 6s | 18 |
| 3: partitionBy + join | | | 63.3 MiB | **185.6 KiB** | 3s | 18 | 
| 4: reduce by key | | | 185.6 KiB | 11.2 KiB | 1s | 1 |

**Total time**: 1.7 min

No significant changes in the overall execution time were observed.
However, it is possible to see the overall amount of shuffled data during the 3rd stage has been reduced from 1.95 MiB (1999.5 KiB) to 185.6 KiB.
Despite this reduction, the overall performance improvement is limited due to, very probably, the small size of data being processed.

**The overall speedup achieved by the optimizations is around 53%.**